In [387]:
#Heart_attack etection model made by Alvaro de la Maza
#I know there are eaier ways of predicting this, however I wanted to try it with a Neural Network
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import sys


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation , Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy



In [388]:
#import dataset, downloaded from kaggle: https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset
heart_a=pd.read_csv("/Users/alvarodelamaza/Desktop/stuff/heart.csv")
heart_a=heart_a.sample(frac=1)
heart=heart_a.head(200)

In [389]:
#we see the features which have a bigger correlation with the possibility of having a heart attack
heart.corr()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
age,1.000000,-0.185728,-0.092140,0.369727,0.230751,0.078690,-0.099983,-0.372992,0.135523,0.254853,-0.229006,0.332740,0.063426,-0.197160
sex,-0.185728,1.000000,-0.081721,-0.107727,-0.238184,-0.012007,0.004225,-0.020023,0.168959,0.081165,0.040474,0.078383,0.201566,-0.244692
cp,-0.092140,-0.081721,1.000000,0.015017,-0.032622,0.012132,0.107184,0.337710,-0.418007,-0.108432,0.133555,-0.231211,-0.118303,0.435182
trtbps,0.369727,-0.107727,0.015017,1.000000,0.179950,0.200867,-0.130023,-0.038471,0.080927,0.184720,-0.143915,0.145444,0.069883,-0.158991
chol,0.230751,-0.238184,-0.032622,0.179950,1.000000,0.074278,-0.153649,0.003679,0.029903,0.144197,-0.038309,0.130670,0.104000,-0.057247
fbs,0.078690,-0.012007,0.012132,0.200867,0.074278,1.000000,-0.068987,0.011490,0.057852,0.089930,-0.030687,0.213571,-0.057747,-0.049071
restecg,-0.099983,0.004225,0.107184,-0.130023,-0.153649,-0.068987,1.000000,0.065727,-0.059497,-0.065312,0.097999,-0.066286,-0.073142,0.143052
thalachh,-0.372992,-0.020023,0.337710,-0.038471,0.003679,0.011490,0.065727,1.000000,-0.376985,-0.342658,0.453605,-0.289460,-0.094580,0.410567
exng,0.135523,0.168959,-0.418007,0.080927,0.029903,0.057852,-0.059497,-0.376985,1.000000,0.248953,-0.276260,0.084485,0.219307,-0.427980
oldpeak,0.254853,0.081165,-0.108432,0.184720,0.144197,0.089930,-0.065312,-0.342658,0.248953,1.000000,-0.613079,0.287347,0.158836,-0.391047


In [390]:
# we extract the data from the datframe and we select the training set and the test set

cholesterol=heart['chol']
age=heart['age']
sex=heart['sex']
chest_pain=heart['cp']
max_heart_rate=heart['thalachh']
previous_peak=heart['oldpeak']
output=heart['output']

data=np.array([cholesterol,age,sex,chest_pain,max_heart_rate,previous_peak])
data=np.transpose(data)

heart2=heart_a[201:303]

cholesterol2=heart2['chol']
age2=heart2['age']
sex2=heart2['sex']
chest_pain2=heart2['cp']
max_heart_rate2=heart2['thalachh']
previous_peak2=heart2['oldpeak']
output2=heart2['output']
output2=np.array(output2)
test_data=np.array([cholesterol2,age2,sex2,chest_pain2,max_heart_rate2,previous_peak2])
test_data=np.transpose(test_data)


(102, 6) (200, 6)
     age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
155   58    0   0     130   197    0        1       131     0      0.6    1   
12    49    1   1     130   266    0        1       171     0      0.6    2   
239   35    1   0     126   282    0        0       156     1      0.0    2   
195   59    1   0     170   326    0        0       140     1      3.4    0   
95    53    1   0     142   226    0        0       111     1      0.0    2   
..   ...  ...  ..     ...   ...  ...      ...       ...   ...      ...  ...   
272   67    1   0     120   237    0        1        71     0      1.0    1   
137   62    1   1     128   208    1        0       140     0      0.0    2   
68    44    1   1     120   220    0        1       170     0      0.0    2   
133   41    1   1     110   235    0        1       153     0      0.0    2   
208   49    1   2     120   188    0        1       139     0      2.0    1   

     caa  thall  output  
155    

In [391]:
#We separate the output vector in two vectors 

np.set_printoptions(threshold=np.inf)
result0=np.zeros(200)
result1=np.zeros(200)
result02=np.zeros(102)
result12=np.zeros(102)
output=np.array(output)
for i in range(200):
    if output[i]==1:
        result1[i]=1
    if output[i]==0:
        result0[i]=1
for i in range(102):
    if output2[i]==1:
        result12[i]=1
    if output2[i]==0:
        result02[i]=1

results=np.array([result0,result1])
results=np.transpose(results)
results2=np.array([result02,result12])
results2=np.transpose(results2)
results2.shape

(102, 2)

In [375]:
#Here we train several times with different number of epoch until we select the best number
number=15
val_loss=np.zeros(number)
val_acc=np.zeros(number)
for i in range(number):
    
    model= Sequential([
    Dense(units=6,kernel_initializer='normal', input_shape=(6,),activation='relu'),
    Dense(units=6,kernel_initializer='normal' ,activation='relu'),  
    Dense(units=4,kernel_initializer='normal' ,activation='relu'),
    Dense(units=2,kernel_initializer='normal',activation='softmax')])
   
    model.compile(optimizer='adam',
            loss="categorical_crossentropy",
            metrics=['accuracy'])
    model.fit(x=data,
            y=results
           ,epochs=(150))
    
    model.save('heart_attack_model')
    new_model= tf.keras.models.load_model('heart_attack_model')
    predictions= new_model.predict([test_data])

    predict=np.argmax(predictions,axis=1)
    
    for k in range(len(predict)):
        store[i][k]=predict[k]
    

       
    val_loss[i], val_acc[i]=model.evaluate(test_data,results2)

predictions=store[np.argmax(val_acc)]
print(predictions, val_acc[np.argmax(val_acc)])

Epoch 1/150
7/7 [==============================] - 0s 772us/step - loss: 0.6933 - accuracy: 0.4914
Epoch 2/150
7/7 [==============================] - 0s 679us/step - loss: 0.6930 - accuracy: 0.5159
Epoch 3/150
7/7 [==============================] - 0s 1ms/step - loss: 0.6926 - accuracy: 0.5245
Epoch 4/150
7/7 [==============================] - 0s 757us/step - loss: 0.6910 - accuracy: 0.5833
Epoch 5/150
7/7 [==============================] - 0s 1ms/step - loss: 0.6922 - accuracy: 0.5260
Epoch 6/150
7/7 [==============================] - 0s 946us/step - loss: 0.6919 - accuracy: 0.5310
Epoch 7/150
7/7 [==============================] - 0s 944us/step - loss: 0.6912 - accuracy: 0.5371
Epoch 8/150
7/7 [==============================] - 0s 864us/step - loss: 0.6898 - accuracy: 0.5525
Epoch 9/150
7/7 [==============================] - 0s 755us/step - loss: 0.6912 - accuracy: 0.5282
Epoch 10/150
7/7 [==============================] - 0s 902us/step - loss: 0.6906 - accuracy: 0.5320
Epoch 11/150


In [ ]:
#we can see now how our model is working with the test set
print('label    prediction    Accuracy:', (val_acc[np.argmax(val_acc)]))
print('----------------------------------------')


for i in range(len(test_data)):
    print('%3.0f %10.0f' %(output2[i], predictions[i]))

In [426]:
#We define the model into a function
def heart_attack_predictor(cholesterol_,age_,sex_,chest_pain_,max_heart_rate_,previous_peak_):
    data_a=np.array([[cholesterol_],[age_],[sex_],[chest_pain_],[max_heart_rate_],[previous_peak_]])
    dataa=np.transpose(data_a)
    predictions= new_model.predict(dataa)

    predict=np.argmax(predictions,axis=1)
    print(predictions)
    if predict==1:
        print('There is a high possibility that you will have a heart attack, visit your doctor please')
    else:
        print('A priori you are not in danger of having a heart attack, if you have pain, visit your doctor please')
    

In [432]:
#We do the final program
print('-- Cholesterol level')
a=float(input())
print('-- Age')
b=float(input())
print('-- Gender')
print('male---->0 // female ---> 1')
c=float(input())
print('-- Chest pain')
print('low----------------->High')
print('0      1      2       3')
d=float(input())
print('-- Maximun heart rate recorded')
e=float(input())
print('-- How big was the previous peak compared to this one')
print('0----------------->3')
f=float(input())
heart_attack_predictor(a,b,c,d,e,f)

-- Cholesterol level
200
-- Age
67 
-- Gender
male---->0 // female ---> 1
1
-- Chest pain
low----------------->High
0      1      2       3
2
-- Maximun heart rate recorded
234
-- How big was the previous peak compared to this one
0----------------->3
2
[[0.05616866 0.9438314 ]]
There is a high possibility that you will have a heart attack, visit your doctor please


SyntaxError: invalid syntax (<ipython-input-433-8da18c0e16cd>, line 1)